In [1]:
# Ελεγχος version pyspark .. Θέλουμε το 3.4.1
import pyspark
print(pyspark.__version__)

3.4.1


In [2]:
# add java to path .. δεν μπορουσα να βρω αλλον τροπο να τρέξει η spark.
# (θελουμε την Java 17 επειδη μονο η Java 8 , Java 11 και Java 17 δουλευει με το pyspark)
import os
os.environ["JAVA_HOME"] = "C:\\Program Files\\Java\\jdk-17"
os.environ["PATH"] += ";C:\\Program Files\\Java\\jdk-17\\bin"

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, avg, format_number
from pyspark.sql.types import FloatType
import pandas as pd

In [4]:
# Διαβάζομε τα δεδομένα

tour_occ_ninat = "data/tour_occ_ninat.xlsx"

excel_data = pd.read_excel(tour_occ_ninat, header=8)

In [5]:
# Κάνω το excel σε csv για να το βαλω στην spark , επειδη δεν μου δουλευε το spark-excel

excel_data.to_csv("data/excel_data.csv", index=False)

In [6]:
# Φτιάχνουμε ενα spark session
spark = SparkSession.builder.appName("EurostatAnalysis").getOrCreate()

In [7]:
# Διαβαζουμε το csv με την spark
spark_df = spark.read.option("header", "true").csv("data/excel_data.csv")

In [8]:
# Ενα print για να δουμε οτι διαβασαμε σωστα τα δεδομενα
spark_df.show(truncate=False)

+------------------------------------------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|GEO/TIME                                        |2006     |2007     |2008     |2009     |2010     |2011     |2012     |2013     |2014     |2015     |
+------------------------------------------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|Belgium                                         |16039090 |16271311 |16360702 |15451017 |16169676 |16723867 |16432646 |16511721 |17068872 |18852087 |
|Bulgaria                                        |11944694 |12006786 |11791454 |9460922  |10547112 |12426723 |13451440 |14370426 |14077798 |13352281 |
|Czech Republic                                  |20090348 |20610186 |19987022 |17746893 |18365947 |19424839 |21793985 |22144896 |22110112 |23286515 |
|Denmark                                         |9453026  |9327579  |8918197  |8299403  |8981

In [9]:
# Αντικαθηστώ το ":" με 0 για να βγουν σωστά οι μέση όροι
# και οποιοδήποτε αλλο string γινεται 0
# και μετατρέπω τα strings σε float για μετεπιτα υπολογισμούς

year_columns = [col for col in spark_df.columns if col.startswith("20")]
for year_col in year_columns:
    spark_df = spark_df.withColumn(
        year_col,
        when(col(year_col).cast(FloatType()).isNotNull(), col(year_col).cast(FloatType())).otherwise(0.0)
    )

# Ερωτημα 3.1 

In [10]:
# Βρίσκω τον μέσο όρο των στηλών για κάθε χρονο
year_columns = [str(year) for year in range(2007, 2015)]
average_columns = [col(column) for column in year_columns]
average_column = sum(average_columns) / len(average_columns)
result = spark_df.select("GEO/TIME", average_column.alias("AverageTourists")) \
    .groupBy("GEO/TIME").avg("AverageTourists")

In [11]:
result.show(truncate=False)

+------------------------------------------+--------------------+
|GEO/TIME                                  |avg(AverageTourists)|
+------------------------------------------+--------------------+
|Sweden                                    |1.137128175E7       |
|Turkey                                    |2.3396917E7         |
|France                                    |1.18260271E8        |
|Greece                                    |6.0361191E7         |
|Slovakia                                  |4234543.375         |
|Belgium                                   |1.63737265E7        |
|Finland                                   |5450927.375         |
|Malta                                     |7568818.625         |
|Croatia                                   |4.338973875E7       |
|Italy                                     |1.72326804E8        |
|Lithuania                                 |2088391.125         |
|Norway                                    |7966946.375         |
|Spain    

# Ερωτημα 3.2

In [12]:
def compare_tourist_counts(countries):
    # Get the columns representing years
    year_columns = spark_df.columns[1:]  # Exclude the first column which should be the country names

    # Collect the country rows as a list
    data_rows = spark_df.collect()

    # Initialize a dictionary to store the results
    result = {country: [] for country in countries}

    # Compare tourist counts for each selected country against Greece
    for country in countries:
        country_row = [row for row in data_rows if row[0] == country]

        if country_row:
            country_row = country_row[0]
            for idx, year in enumerate(year_columns):
                year_val = int(year)  # Convert column names to integers for comparison
                greece_val = int([row for row in data_rows if row[0] == "Greece"][0][idx + 1])

                if country_row[idx + 1] < greece_val:  # Compare if the country value is less than Greece
                    result[country].append(year_val)

    return result

In [13]:

# Έβαλα τις πρώτες 5 χώρες

selected_countries = ["Belgium", "Bulgaria", "Czech Republic", "Denmark", "Germany (until 1990 former territory of the FRG)"]
result = compare_tourist_counts(selected_countries)

# Print the years where Greece had higher tourist counts than the selected countries
for country, years in result.items():
    if len(years) > 0:
        print(f"Για {len(years)} χρόνια, Η Ελλάδα είχε αριθμό διανυκτερεύσεων μεγαλύτερο απο αυτην την χώρα '{country}' , Οι συγκεκριμένες χρονιές: {years}")
    else:
        print(f"Η Ελλάδα δεν ειχε ποτε μεγαλύτερο αριθμό διανυκτερεύσεων απο την χώρα '{country}'")

Για 10 χρόνια, Η Ελλάδα είχε αριθμό διανυκτερεύσεων μεγαλύτερο απο αυτην την χώρα 'Belgium' , Οι συγκεκριμένες χρονιές: [2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]
Για 10 χρόνια, Η Ελλάδα είχε αριθμό διανυκτερεύσεων μεγαλύτερο απο αυτην την χώρα 'Bulgaria' , Οι συγκεκριμένες χρονιές: [2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]
Για 10 χρόνια, Η Ελλάδα είχε αριθμό διανυκτερεύσεων μεγαλύτερο απο αυτην την χώρα 'Czech Republic' , Οι συγκεκριμένες χρονιές: [2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]
Για 10 χρόνια, Η Ελλάδα είχε αριθμό διανυκτερεύσεων μεγαλύτερο απο αυτην την χώρα 'Denmark' , Οι συγκεκριμένες χρονιές: [2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]
Για 2 χρόνια, Η Ελλάδα είχε αριθμό διανυκτερεύσεων μεγαλύτερο απο αυτην την χώρα 'Germany (until 1990 former territory of the FRG)' , Οι συγκεκριμένες χρονιές: [2009, 2011]
